In [1]:
import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import h5py
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]

vmec_file = vmec_files[1]
import StellBlanket.SBGeom as SBGeom
from functools import partial
from jax_sbgeom.flux_surfaces.flux_surfaces_base import ToroidalExtent, _cylindrical_position_interpolated, _interpolate_array
from jax_sbgeom.flux_surfaces.flux_surfaces_extended import FluxSurfaceNormalExtended
#from jax_sbgeom.flux_surfaces.flux_surface_meshing import _mesh_watertight_surfaces_connectivity
from tests.flux_surfaces.test_flux_surface_base import test_position, _mesh_to_pyvista_mesh, test_watertight_surfaces, _check_vectorized
import jax_sbgeom.flux_surfaces.flux_surface_meshing as fsm
import pyvista as pv
from dataclasses import dataclass

Triangle Elements not compiled


In [2]:
def _get_flux_surfaces(vmec_file):
    fs_jax    = jsb.flux_surfaces.FluxSurface.from_hdf5(vmec_file)
    fs_sbgeom = SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file)
    return fs_jax, fs_sbgeom



def _get_extended_flux_surfaces(vmec_file):    
    fs_jax    = jsb.flux_surfaces.FluxSurfaceNormalExtended.from_hdf5(vmec_file)
    fs_sbgeom = SBGeom.Flux_Surfaces_Normal_Extended(SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file))
    return fs_jax, fs_sbgeom
fs_jax, fs_sbgeom = _get_extended_flux_surfaces(vmec_file)  # just to compile

In [3]:
ss, tt = jnp.meshgrid(jnp.array([1.5, 1.6]), jnp.linspace(0, 2 * jnp.pi, 5), indexing='ij')
fs_jax.cartesian_position(1.5, ss, tt)

Array([[[ 23.34255765,  -0.04975456,   3.54180509],
        [ -0.08825212,  21.91470157,   3.75062059],
        [-21.46185664,  -0.11860545,   1.76577635],
        [ -0.0501968 , -23.13289925,   1.65464056],
        [ 23.34255765,  -0.04975456,   3.54180509]],

       [[ 23.0713958 ,  -0.08611057,   3.60067526],
        [ -0.07539242,  21.66266275,   3.85208243],
        [-21.21780272,  -0.13108573,   1.87432829],
        [ -0.04671122, -23.02121853,   1.55113451],
        [ 23.0713958 ,  -0.08611057,   3.60067526]]], dtype=float64)

In [7]:
from jax_sbgeom.flux_surfaces.flux_surfaces_extended import FluxSurfaceNormalExtended, _normal_extended_principal_curvatures, _normal_extended_principal_curvatures_jit
_normal_extended_principal_curvatures(fs_jax.data, fs_jax.settings, 1.5, ss, tt)

[[[ 0.03936199  1.01773538]
  [ 0.05314682  0.50113663]
  [-0.13579406  0.06603556]
  [-0.49497958 -0.03873364]
  [ 0.03936199  1.01773538]]

 [[ 0.02356461  1.33404579]
  [ 0.05116669  0.72522694]
  [-0.08582024  0.11588139]
  [-0.4978989  -0.04412146]
  [ 0.02356461  1.33404579]]]
[[[ 0.04015222  2.07222241]
  [ 0.05459766  0.66868888]
  [-0.12716026  0.06829036]
  [-0.39678046 -0.03799775]
  [ 0.04015222  2.07222241]]

 [[ 0.02384557  4.00641898]
  [ 0.05251007  1.13781341]
  [-0.0822892   0.12300859]
  [-0.39865417 -0.04316912]
  [ 0.02384557  4.00641898]]]


Array([[[ 0.03936199,  1.01773538],
        [ 0.05314682,  0.50113663],
        [-0.13579406,  0.06603556],
        [-0.49497958, -0.03873364],
        [ 0.03936199,  1.01773538]],

       [[ 0.02356461,  1.33404579],
        [ 0.05116669,  0.72522694],
        [-0.08582024,  0.11588139],
        [-0.4978989 , -0.04412146],
        [ 0.02356461,  1.33404579]]], dtype=float64)

In [8]:
fs_jax.cartesian_position(1.5, ss, tt)

Array([[[ 23.34255765,  -0.04975456,   3.54180509],
        [ -0.08825212,  21.91470157,   3.75062059],
        [-21.46185664,  -0.11860545,   1.76577635],
        [ -0.0501968 , -23.13289925,   1.65464056],
        [ 23.34255765,  -0.04975456,   3.54180509]],

       [[ 23.0713958 ,  -0.08611057,   3.60067526],
        [ -0.07539242,  21.66266275,   3.85208243],
        [-21.21780272,  -0.13108573,   1.87432829],
        [ -0.04671122, -23.02121853,   1.55113451],
        [ 23.0713958 ,  -0.08611057,   3.60067526]]], dtype=float64)

In [ ]:
fs_jax.principal_curvatures(1.5, ss, tt)

JitTracer<float64[2,5,2]>


Array([[[ 0.03936199,  1.01773538],
        [ 0.05314682,  0.50113663],
        [-0.13579406,  0.06603556],
        [-0.49497958, -0.03873364],
        [ 0.03936199,  1.01773538]],

       [[ 0.02356461,  1.33404579],
        [ 0.05116669,  0.72522694],
        [-0.08582024,  0.11588139],
        [-0.4978989 , -0.04412146],
        [ 0.02356461,  1.33404579]]], dtype=float64)